In [51]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

BORDER = "==============================================================="

In [52]:
def load_data(filename):
    raw_text = open(filename).read()
    text_list = raw_text.strip().split('\n')
    return text_list

In [53]:
def remove_int(text):
    '''
    Remove numbers from strings
    '''
    new_text = []
    for line in text:
        # Remove integer values
        no_digits = ''.join([i for i in line if not i.isdigit()])

        # Remove punctuation
        new_text.append(no_digits)
    return new_text

In [54]:
def remove_empty(text):
    '''
    Removes all empty string
    '''
    new_text = []
    for line in text:
        if line != '':
            new_text.append(line)
    return new_text

In [55]:
def process_data_RNN(text):
    '''
    Create fixed length training sequences of length 40 char from the sonnet
    corpus.

    Input: Text file in the form of list of words

    Output: X, Y, dataX, dataY, int_to_char, n_vocab
    '''
    
    new_text_list = remove_int(text)
    new_text_list = remove_empty(new_text_list)
    new_text = '\n'.join(new_text_list)
    
    print(BORDER)
    print("Processed text: ", new_text)
    print(BORDER)

    # create mapping of unique chars to integers, and a reverse mapping
    chars = sorted(list(set(new_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    # summarize the loaded data
    n_chars = len(new_text)
    n_vocab = len(chars)
    print("Total Characters: ", n_chars)
    print("Total Vocab: ", n_vocab)
    print("Int to char: ", int_to_char)

    # prepare the dataset of input to output pairs encoded as integers
    seq_length = 40
    dataX = []
    dataY = []
    for i in range(0, n_chars - seq_length):
        seq_in = new_text[i:i + seq_length]
        seq_out = new_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)
    print("Total Patterns: ", n_patterns)
    print(BORDER)

    X = np.zeros((n_patterns, seq_length, n_vocab))
    y = np.zeros((n_patterns, n_vocab))
    for i, sentence in enumerate(dataX):
        for t, ind in enumerate(sentence):
            X[i, t, ind] = 1
        y[i, dataY[i]] = 1


    return X, y, dataX, dataY, int_to_char, char_to_int

In [56]:
def train_LSTM(X, y, verbose=0):
    '''
    Takes training data X and Y and returns the fitted LSTM model

    Input:
        X : a list of sequences of int
        Y : one-hot encoding of the int coming after the sequence
    '''

    # Take a submit of sequences
#     X = X[0::50]
#     y = y[0::50]

    # define the LSTM model
    model = Sequential()
    model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

    # fit the model
    model.fit(X, y, epochs=25, batch_size=128, verbose=verbose)
    return model

In [63]:
def generate_text(model, dataX, int_to_char, verbose=0):
    '''
    Given model, dataX, int_to_char, n_vocab returns generated_text using
    predict function

    Input:
        model: the LSTM model that we trained
        dataX: list of sequences
        int_to_char: a dictionary matching interger to specific character
        n_vocab: number of unique characters we have

    Output: generate_text as string

    '''
    print(BORDER)
    print("Generating text")
    n_vocab = len(int_to_char)

    # pick a random seed
    start = np.random.randint(0, len(dataX) - 1)
    pattern = dataX[start]
    seq = [int_to_char[value] for value in pattern]
    size = len(pattern)
    
    print("Seed: ", ''.join(seq))

    # generate characters
    for i in range(600):
        x = np.zeros((1, size, n_vocab))
        for t, char in enumerate(pattern):
            x[0, t, char] = 1

        # Make prediction using trained model
        prediction = model.predict(x, verbose=verbose)
        print("pred: ", prediction[0])
        index = np.argmax(prediction[0])
        print("selected index: ", index)

        # Convert prediction to character
        result = int_to_char[index]
        print("selected char: ", result)

        # Add prediction to pattern and set to size 40
        pattern.append(index)
        pattern = pattern[1:1 + size]
        print("new pattern: ", ''.join([int_to_char[value] for value in pattern]))

        # Add result to seq
        seq.append(result)

    # Return seq as string
    return ''.join(seq)

In [58]:
def save_textfile(filename, text):
    '''
    Given filename and text, save text in file

    Input: filename and text as string
    '''
    f = open(filename, 'w')
    f.write(text)
    f.close()
    return 0

In [59]:
file = 'data/shakespeare.txt'
save = 'generated/shakespeare.txt'

text_list = load_data(file)
(X, y, dataX, dataY, int_to_char, char_to_int) = process_data_RNN(text_list)

Processed text:  From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
  Pity the world, or else this glutton be,
  To eat the world's due, by the grave and thee.
                   
When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a tattered weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say within thine own deep sunken eyes,
Were an all-eating shame, and thriftless prai

Total Patterns:  97309


In [60]:

model = train_LSTM(X, y, verbose=1)

Epoch 1/25
97309/97309 [==============================] - 113s 1ms/step - loss: 2.6896 - acc: 0.2685
Epoch 2/25
97309/97309 [==============================] - 121s 1ms/step - loss: 2.1603 - acc: 0.3804
Epoch 3/25
97309/97309 [==============================] - 129s 1ms/step - loss: 1.9999 - acc: 0.4192
Epoch 4/25
97309/97309 [==============================] - 129s 1ms/step - loss: 1.9064 - acc: 0.4430
Epoch 5/25
97309/97309 [==============================] - 126s 1ms/step - loss: 1.8381 - acc: 0.4586
Epoch 6/25
97309/97309 [==============================] - 125s 1ms/step - loss: 1.7851 - acc: 0.4730
Epoch 7/25
97309/97309 [==============================] - 119s 1ms/step - loss: 1.7418 - acc: 0.4839
Epoch 8/25
97309/97309 [==============================] - 117s 1ms/step - loss: 1.7044 - acc: 0.4941
Epoch 9/25
97309/97309 [==============================] - 134s 1ms/step - loss: 1.6710 - acc: 0.5026
Epoch 10/25
97309/97309 [==============================] - 179s 2ms/step - loss: 1.6409 - a

In [64]:
generated = generate_text(model, dataX, int_to_char, verbose=1)
save_textfile(save, generated)
print(generated)

Generating text
Seed:   do see.
Save that my soul's imaginary s
1/1 [==============================] - 1s 672ms/step
pred:  [5.7794364e-07 1.4105239e-03 4.7816428e-11 7.4599881e-04 1.1907832e-11
 2.4703340e-11 6.8866444e-05 3.5257324e-09 9.7170464e-08 2.6199969e-09
 1.1368148e-11 3.2043049e-10 3.7732756e-10 1.1511156e-10 8.3541146e-11
 3.1742230e-11 4.2187812e-10 8.7447237e-11 2.6832283e-09 2.4716111e-11
 5.3079993e-06 3.4313175e-08 3.5845808e-08 4.9280613e-10 1.3736171e-10
 2.2049365e-10 1.3110270e-10 1.0915170e-09 6.2141097e-09 3.0369054e-11
 6.7928715e-07 2.4342775e-10 2.3988267e-08 9.6768915e-10 3.2851202e-10
 4.0840425e-02 1.5501387e-03 2.5243862e-02 5.6465196e-05 8.7023407e-02
 8.2518649e-04 1.2261891e-02 2.8107452e-01 1.2092382e-01 8.5787525e-07
 1.9813571e-03 1.1081430e-02 1.2787383e-03 9.5927047e-05 7.0407383e-02
 1.7761242e-01 1.0527695e-07 3.6858369e-04 7.6972705e-04 7.7589087e-02
 4.3267053e-02 9.2834009e-05 4.2890660e-02 2.0621162e-06 5.2990980e-04
 6.2655006e-09]
selected

pred:  [4.97995550e-03 4.39901778e-04 5.88827334e-06 1.22183748e-03
 8.38252672e-05 6.26813016e-06 2.61664134e-03 2.33002138e-05
 1.43508974e-03 1.13328228e-04 7.19390709e-06 6.14758028e-05
 1.10531560e-04 3.24385510e-05 1.37981551e-05 1.83027169e-05
 1.02446520e-05 3.97176373e-05 7.45979150e-06 1.77797410e-05
 1.55658927e-04 5.86045843e-08 4.07918378e-07 3.47554451e-05
 1.12955327e-04 8.04003048e-06 2.78478510e-05 1.51190861e-05
 9.18035676e-06 3.16024889e-05 3.42060433e-04 1.25419347e-05
 7.62191732e-08 7.46702353e-05 9.05974139e-06 3.81839368e-03
 4.41188477e-02 4.84049842e-02 5.34873344e-02 5.16488254e-02
 1.06746539e-01 3.94363627e-02 4.17156927e-02 1.57790799e-02
 4.91810171e-03 1.30451452e-02 6.27941862e-02 4.01751883e-02
 2.99506285e-03 2.83476133e-02 6.11829236e-02 5.27278869e-04
 3.55582312e-02 1.26227885e-01 4.38673496e-02 2.52254074e-03
 5.64235114e-02 1.03639364e-01 5.04698619e-07 5.38997410e-04
 1.06142124e-06]
selected index:  53
selected char:  s
new pattern:  Save that

pred:  [5.99213969e-03 4.46599871e-01 5.86964852e-05 1.76288420e-03
 9.82096186e-04 1.82006479e-04 1.31675526e-02 4.28150117e-04
 2.15093559e-03 2.13889149e-03 1.92123887e-04 1.18726038e-03
 9.20139253e-02 2.70659588e-02 3.56312143e-03 3.61344195e-03
 1.26140099e-03 2.74925064e-02 2.16601082e-04 1.48853650e-02
 4.04410437e-02 2.71271716e-07 3.74318756e-06 6.40435284e-03
 1.13122817e-02 1.43644447e-02 2.71087065e-02 1.69147807e-03
 2.63922819e-04 2.84714270e-02 9.09348130e-02 5.84189082e-04
 2.02427572e-07 5.69505580e-02 2.93179089e-03 3.96978157e-03
 1.75763993e-03 2.58817244e-03 3.61865415e-04 2.00665626e-03
 1.07129081e-03 2.53911992e-03 5.04520442e-03 3.83859547e-03
 1.91998042e-04 3.19701940e-04 1.04224756e-02 4.35378635e-03
 5.22987777e-03 2.37333542e-03 2.61013652e-03 1.36557268e-04
 1.70709379e-03 3.88776069e-03 1.37514398e-02 5.70582401e-04
 1.12527945e-04 3.30385752e-03 3.80065874e-04 1.05051906e-03
 1.78523419e-06]
selected index:  1
selected char:   
new pattern:  t my soul'

1/1 [==============================] - 0s 6ms/step
pred:  [1.05610169e-07 1.04273728e-03 2.29453678e-11 4.84254451e-05
 2.09734116e-14 5.09358267e-12 2.05210745e-05 2.23665347e-06
 1.05661400e-08 1.30137748e-10 5.46555466e-12 1.49745286e-11
 1.02790025e-11 9.38673618e-13 4.27194819e-13 3.99369631e-14
 3.31629147e-12 3.65700309e-13 2.92419838e-10 1.19312166e-12
 1.53823834e-08 2.70124634e-09 1.60212144e-09 3.12498465e-12
 3.55661111e-13 2.04651563e-13 1.99868862e-13 7.56905823e-13
 2.31517236e-11 7.88358582e-13 1.54541375e-08 1.21171649e-12
 1.31472577e-09 2.89782196e-12 2.78342952e-12 6.96522929e-03
 5.50661644e-04 5.65764349e-05 1.81196683e-05 8.77903774e-03
 1.32133905e-03 3.07560094e-06 7.96794920e-05 1.92676038e-02
 8.08636536e-09 6.73340328e-05 2.87974551e-02 4.30334127e-03
 3.37008275e-02 6.99067861e-02 1.38269213e-03 1.02906705e-08
 6.35001719e-01 1.33769009e-02 5.10490034e-03 1.63303420e-01
 2.59740569e-04 3.69535503e-03 2.05140213e-06 2.94200028e-03
 2.08479803e-10]
selected i

 3.03933372e-11]
selected index:  54
selected char:  t
new pattern:  ginary show the strong,
  The world that
1/1 [==============================] - 0s 7ms/step
pred:  [3.9676155e-07 9.9391770e-01 3.0466241e-10 1.4580398e-04 2.1334559e-14
 2.4936008e-07 2.5451267e-03 4.0566047e-06 4.7823933e-06 8.0918664e-07
 2.4181153e-08 6.4091438e-08 6.8791998e-14 1.1394894e-14 8.4933173e-14
 5.8262064e-15 6.9587402e-13 1.6783995e-14 5.0920997e-11 1.4505626e-14
 1.8064003e-11 1.2115074e-10 3.0524233e-10 3.4270252e-14 2.2542083e-14
 1.5967870e-15 1.2005967e-14 3.4873946e-13 1.0896228e-11 4.2115638e-16
 1.1450037e-10 9.1898657e-13 1.2257649e-10 8.7327078e-14 3.9595727e-14
 2.1246698e-05 3.9473861e-07 3.0096129e-05 1.0396516e-06 2.1902628e-03
 1.1511632e-05 7.5201447e-06 3.7065204e-04 3.3862607e-04 1.7306494e-08
 8.6984401e-06 1.6385249e-05 2.7859878e-06 7.1533473e-06 1.4707512e-04
 5.5953014e-06 5.2505995e-09 1.2897077e-05 5.6033354e-05 2.8603743e-06
 6.8107736e-05 3.1601743e-07 6.4402419e-05 1.811123

...

1/1 [==============================] - 0s 6ms/step
pred:  [9.05117037e-10 3.10375422e-01 5.65024921e-16 2.58228265e-05
 7.92651396e-21 6.47141973e-17 6.89924354e-06 2.03361963e-11
 2.13879872e-10 3.79311437e-13 1.48451189e-18 1.79854622e-14
 9.97631194e-20 1.29265487e-22 8.81208573e-22 8.25818329e-24
 1.13736720e-20 4.48478254e-22 3.58047762e-16 9.79021165e-22
 3.30817351e-13 6.39046371e-10 3.85252594e-12 7.99923784e-20
 1.04364657e-21 2.21435149e-22 3.27313473e-23 5.29969004e-21
 1.89031549e-17 2.73000248e-22 5.09201893e-13 3.89047987e-20
 6.21700080e-10 7.51808303e-21 2.86725802e-21 1.13040926e-02
 7.06860916e-08 5.53352002e-04 2.50852281e-06 1.92089397e-02
 9.63811690e-05 2.02721840e-05 2.90375971e-03 2.05711517e-02
 3.59210833e-15 4.24892193e-07 6.34853379e-04 4.15497180e-03
 3.86786193e-01 2.01740855e-04 1.05899790e-05 7.68489615e-14
 1.54611513e-01 5.80292977e-02 4.85852454e-03 8.88150069e-04
 6.20863557e-06 1.13366965e-04 3.55103509e-07 2.46352423e-02
 1.11658517e-14]
selected i

1/1 [==============================] - 0s 8ms/step
pred:  [1.4577119e-05 1.6598953e-02 2.0573762e-07 4.3127132e-03 1.3590975e-12
 1.0566167e-08 1.3841258e-04 4.2140688e-05 9.3982153e-06 1.4899547e-06
 1.2494586e-07 5.5934885e-07 2.1173030e-10 2.5398903e-11 1.8241225e-10
 1.3592200e-11 2.4834459e-10 6.0123326e-12 2.7107516e-08 1.6452588e-11
 7.1108309e-07 7.8527898e-07 1.2664959e-06 4.1754183e-10 1.4382782e-11
 1.5299075e-10 2.3164370e-11 1.1230658e-10 1.5617392e-08 2.5423476e-11
 1.2659416e-06 3.3956660e-10 7.4946212e-07 5.6223302e-09 9.1028296e-11
 1.2457739e-03 2.9280953e-02 7.2360244e-03 2.5015803e-02 4.9084157e-04
 6.8186880e-03 2.3211142e-02 1.3518215e-03 6.4428635e-03 1.0576471e-06
 5.3086044e-04 6.1731111e-02 3.9231028e-02 3.5191473e-02 1.3884555e-03
 9.0426328e-03 5.2569771e-07 5.2172118e-01 1.3601291e-01 2.1840729e-02
 1.6362699e-02 8.0723222e-03 1.2774894e-02 1.2641157e-05 1.3868034e-02
 4.4323627e-08]
selected index:  52
selected char:  r
new pattern:  rt thee show the stron

1/1 [==============================] - 0s 7ms/step
pred:  [9.3394640e-08 2.3736988e-04 2.1361135e-10 3.6892798e-04 1.2820875e-11
 1.9720611e-11 1.4161423e-04 2.8878501e-08 9.3070824e-08 1.4701611e-08
 1.1814641e-10 5.7920679e-10 5.2492216e-10 4.4563821e-11 1.3519612e-10
 2.6985228e-11 8.1376467e-10 1.5723691e-11 2.4842113e-09 1.8165849e-11
 4.5233413e-05 2.0380549e-08 2.3643022e-08 1.5586792e-10 1.4141595e-10
 1.0053019e-10 1.6418311e-10 2.3817345e-10 9.6974284e-10 2.1144177e-11
 2.1513838e-06 1.6027736e-10 2.4332472e-08 5.4276930e-09 1.1433487e-10
 8.8908598e-02 6.7629740e-03 6.0953898e-03 3.2812225e-06 8.1084549e-02
 1.1175862e-03 2.6477216e-04 4.7955845e-02 5.3753849e-02 1.8041328e-08
 1.8910217e-04 5.6234032e-02 7.5992630e-03 1.9940853e-04 6.9579147e-02
 4.2161603e-02 2.1742412e-08 1.6201065e-03 3.4430234e-03 2.0283718e-01
 2.1352760e-01 5.7716315e-05 1.1556325e-01 3.5920552e-06 2.4246769e-04
 3.8275622e-11]
selected index:  55
selected char:  u
new pattern:  show the stronge,
  Th

1/1 [==============================] - 0s 7ms/step
pred:  [4.46176950e-11 6.71101850e-04 1.31617406e-16 5.73334432e-07
 2.84763108e-25 2.78575504e-17 1.54651048e-07 1.43273596e-11
 1.29842179e-11 1.53145386e-14 4.71080548e-19 1.03129325e-16
 5.08235197e-21 1.46453479e-22 1.15036121e-20 2.43828676e-22
 2.54033913e-19 2.51196562e-22 3.39238665e-15 3.03751964e-22
 3.43815253e-16 1.61093985e-10 8.53745633e-12 1.63672063e-21
 9.41920621e-23 3.75596190e-22 1.12815676e-23 3.49334076e-20
 9.85942433e-16 8.66890529e-24 3.46443273e-15 2.46563143e-18
 1.23680302e-10 2.08245385e-21 1.05836047e-20 7.89203386e-06
 3.34696125e-07 6.46067565e-06 2.74718161e-07 2.44144816e-04
 1.65005258e-05 4.83245302e-08 2.13988878e-05 1.09387584e-04
 3.45825320e-15 1.95491907e-06 7.17860385e-05 1.19187916e-03
 1.36355881e-03 7.25395745e-04 2.83700720e-05 8.31705717e-14
 6.84323430e-04 1.55852484e-02 1.97359105e-03 9.77035284e-01
 1.85273461e-06 1.90841936e-04 1.84105487e-12 6.76903510e-05
 2.19540166e-14]
selected i

pred:  [1.60837473e-07 3.55482734e-05 2.91510815e-07 1.43377576e-03
 1.03065828e-11 6.52482686e-06 1.73434764e-05 7.23433041e-05
 4.19712933e-06 3.15068382e-06 9.02013468e-08 1.99821784e-07
 9.95894773e-12 3.91779924e-12 1.88812160e-11 6.88810684e-12
 1.05626806e-10 5.38225226e-13 8.86025386e-09 3.46360397e-12
 2.97493870e-08 1.93829770e-08 4.70046295e-08 1.51732203e-11
 1.00143732e-12 1.61387743e-12 2.84472793e-12 1.04637632e-10
 1.60797131e-09 1.17765444e-12 6.86736401e-09 1.17527030e-10
 2.22511520e-08 4.50089532e-12 1.08128732e-11 1.54945627e-01
 1.07992919e-05 1.00869674e-05 5.55019255e-07 4.17228103e-01
 1.48815061e-05 3.33799449e-06 3.01051914e-05 1.48802802e-01
 3.98252013e-07 2.85559418e-05 1.02672508e-04 6.56448246e-05
 4.74493245e-05 1.27537325e-01 2.09487234e-06 3.33007250e-07
 5.73046133e-03 3.19993637e-06 2.24941716e-04 5.28035173e-03
 1.74708271e-06 1.01432692e-04 5.45603918e-08 1.38253286e-01
 3.73168474e-09]
selected index:  39
selected char:  e
new pattern:  ,
  Then 